In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ziadadelkhalifa","key":"377429fc9f6e3877774c2ceff6182ded"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
#Downloading the data from kaggle

!kaggle datasets download -d haithemhermessi/sanad-dataset

Dataset URL: https://www.kaggle.com/datasets/haithemhermessi/sanad-dataset
License(s): U.S. Government Works
 96% 63.0M/65.7M [00:04<00:00, 22.9MB/s]
100% 65.7M/65.7M [00:04<00:00, 15.8MB/s]


In [4]:
!unzip /content/sanad-dataset.zip -d text-data

Streaming output truncated to the last 5000 lines.
  inflating: text-data/Tech/1500.txt  
  inflating: text-data/Tech/1501.txt  
  inflating: text-data/Tech/1502.txt  
  inflating: text-data/Tech/1503.txt  
  inflating: text-data/Tech/1504.txt  
  inflating: text-data/Tech/1505.txt  
  inflating: text-data/Tech/1506.txt  
  inflating: text-data/Tech/1507.txt  
  inflating: text-data/Tech/1508.txt  
  inflating: text-data/Tech/1509.txt  
  inflating: text-data/Tech/1510.txt  
  inflating: text-data/Tech/1511.txt  
  inflating: text-data/Tech/1512.txt  
  inflating: text-data/Tech/1513.txt  
  inflating: text-data/Tech/1514.txt  
  inflating: text-data/Tech/1515.txt  
  inflating: text-data/Tech/1516.txt  
  inflating: text-data/Tech/1517.txt  
  inflating: text-data/Tech/1518.txt  
  inflating: text-data/Tech/1519.txt  
  inflating: text-data/Tech/1520.txt  
  inflating: text-data/Tech/1521.txt  
  inflating: text-data/Tech/1522.txt  
  inflating: text-data/Tech/1523.txt  
  inflating: 

In [5]:
#Collect all the text files from all the folders and make them in one text file


import os

# Directory containing the folders with text files
base_directory = r'/content/text-data'

# Output file for combined content
output_file = os.path.join(base_directory, '/content/combined_text.txt')

# Open the output file in write mode
with open(output_file, 'w', encoding='utf-8') as outfile:
    # Loop through each folder in the base directory
    for folder_name in sorted(os.listdir(base_directory)):
        folder_path = os.path.join(base_directory, folder_name)

        # Check if the path is a directory
        if os.path.isdir(folder_path):
            # Write folder name as a section header
            outfile.write(f'\n\n=== Content from Folder: {folder_name} ===\n\n')

            # Loop through all text files in the folder
            for filename in sorted(os.listdir(folder_path)):
                if filename.endswith('.txt'):
                    file_path = os.path.join(folder_path, filename)
                    with open(file_path, 'r', encoding='utf-8') as infile:
                        # Write the content of each file to the output file
                        outfile.write(infile.read())
                        outfile.write("\n")  # Add a newline to separate file contents

print(f'All text files have been concatenated into {output_file}')


All text files have been concatenated into /content/combined_text.txt


In [13]:
!pip install sentence_transformers
!pip install langchain
!pip install langchain_community
!pip install chromadb
!pip install langchain_huggingface
!pip install langchain_chroma
!pip install langchain_google_genai
!pip install langchain_core

In [1]:
#Applaying chunks in the text file that contain all my data

# Path to the text file
text_file_path = r'/content/combined_text.txt'

from langchain.text_splitter import RecursiveCharacterTextSplitter

# Define a basic Document class with page_content and metadata
class Document:
    def __init__(self, page_content, metadata=None):
        self.page_content = page_content
        self.metadata = metadata if metadata is not None else {}



# Read the content of the text file
with open(text_file_path, 'r', encoding='utf-8') as file:
    text_content = file.read()

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3450, chunk_overlap=250)

# Create a list of Document objects with metadata
documents = [Document(page_content=text_content, metadata={"source": text_file_path})]

# Split the text into smaller chunks
split_documents = text_splitter.split_documents(documents)


print(len(split_documents))



40593


In [ ]:
#create the embedding model


from langchain_huggingface import HuggingFaceEmbeddings

model_name = "Omartificial-Intelligence-Space/GATE-AraBert-v1"
embeder = HuggingFaceEmbeddings(
    model_name=model_name
)

from langchain_chroma import Chroma
persist_directory = '/content/db22'
vec_db = Chroma.from_documents(
    split_documents,
    embeder,
    persist_directory=persist_directory
)


In [19]:
#load the embedding model

persist_directory = '/content/db22'
db2 = Chroma(
    embedding_function=embeder,
    persist_directory=persist_directory
)

retriever = db2.as_retriever(
    search_type="similarity", search_kwargs={"k": 5}
)




#Connect to drive and save the model in specific folder there to call it in the in the interface

In [21]:
#https://drive.google.com/drive/folders/1ckxTscOs1xrS_rL9j5-lBTiZLk2fU446?usp=sharing

# Step 1: Install PyDrive
!pip install -U -q PyDrive

# Step 2: Import necessary libraries
from google.colab import drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os

# Step 3: Mount Google Drive
drive.mount('/content/drive')


# Define the path where you want to save the file
#foler_path = '/content/drive/MyDrive/vector_db'

import shutil

# Path to the folder you want to compress
folder_path = '/content/db22'

# Path where you want to save the zip file
zip_path = '/content/db22.zip'

# Compress the folder into a zip file
shutil.make_archive('/content/db22', 'zip', folder_path)


# Define the destination path in Google Drive
drive_path = '/content/drive/MyDrive/database_db2'

# Move the zip file to Google Drive
shutil.move(zip_path, drive_path)

print(f'Folder saved to {drive_path}')


Folder saved to /content/drive/MyDrive/database_db2


In [12]:
#second embedding model just to find the best model to get good result

from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeder = HuggingFaceEmbeddings(
    model_name=model_name
)

from langchain_chroma import Chroma
persist_directory = '/content/db'
vec_db = Chroma.from_documents(
    split_documents,
    embeder,
    persist_directory=persist_directory
)


Saving second embedding model

In [19]:

#https://drive.google.com/drive/folders/1ckxTscOs1xrS_rL9j5-lBTiZLk2fU446?usp=sharing



# Define the path where you want to save the file
#foler_path = '/content/drive/MyDrive/vector_db'

import shutil

# Path to the folder you want to compress
folder_path = '/content/db'

# Path where you want to save the zip file
zip_path = '/content/db.zip'

# Compress the folder into a zip file
shutil.make_archive('/content/db', 'zip', folder_path)


import shutil

# Define the destination path in Google Drive
drive_path = '/content/drive/MyDrive/database_dp2'

# Move the zip file to Google Drive
shutil.move(zip_path, drive_path)

print(f'Folder saved to {drive_path}')



'/content/db.zip'

In [21]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma


# Define the prompt template with a focus on context in Arabic
template = '''
You are an AI chatbot developed by humans, and your task is to answer user questions based on the knowledge provided in the context below.
The question will be in Arabic, and your answer should be in Arabic as well.
The provided context contains information about various categories: [Culture, Finance, Medicine, Politics, Religion, Sports, and Technology].
Try to answer using the information given in the context below. If the answer is not clearly available in the context, follow these steps:
    1. Apologize to the user.
    2. Inform the user that you do not know the answer to the question.
    3. Ask the user if they have any more questions.
At the end of the answer, thank the user and ask if they have any other questions.
Your answer should be detailed and at least 300 words long.
If asked about yourself, tell the user that you are a chatbot developed by humans and not a human.

Knowledge you have:
{context}

Question: {question}

Answer:
'''



def format_docs(docs):
    return "\n\n".join(docs)  # Assuming 'docs' is a list of strings

# Initialize the AraGPT2 model
aragpt2_model_name = "aubmindlab/aragpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(aragpt2_model_name)
aragpt2_model = GPT2LMHeadModel.from_pretrained(aragpt2_model_name)

# Function to generate a response using AraGPT2
def generate_aragpt2_response(context, question):
    input_text = f"{context}\n\nالسؤال: {question}\nالإجابة:"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    output = aragpt2_model.generate(input_ids, max_length=500, num_return_sequences=1)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

# Create a custom RAG prompt
custom_rag_prompt = PromptTemplate.from_template(template)

# Define a dummy RAG chain for demonstration
def rag_chain(context, question):
    formatted_context = format_docs([context])  # Manually format the context
    answer = generate_aragpt2_response(formatted_context, question)
    return answer

# Example context to test
example_context = "هنا يوجد مثال على السياق الذي يحتوي على معلومات ذات صلة بسؤال المستخدم."

# Question to ask
question = 'اقترب نادي مانشستر يونايتد من ضم مدافع نادي فولهام كريس سمولينغ (20 عاماً) مقابل 8 ملايين جنيه بعد موافقة إدارة النادي على بيعه نهاية الموسم الحالي.'

# Generate the final answer using AraGPT2
answer = rag_chain(example_context, question)
print(answer)

vocab.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.52M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


هنا يوجد مثال على السياق الذي يحتوي على معلومات ذات صلة بسؤال المستخدم.

السؤال: اقترب نادي مانشستر يونايتد من ضم مدافع نادي فولهام كريس سمولينغ (20 عاماً) مقابل 8 ملايين جنيه بعد موافقة إدارة النادي على بيعه نهاية الموسم الحالي.
الإجابة: بعد أن كان قد انتقل إلى نادي مانشستر يونايتد في صيف عام 2007 ، أصبح اللاعب الإنجليزي الشاب ( 20 عاما ) ، الذي يلعب في مركز الجناح الأيمن ، أحد أبرز اللاعبين في الدوري الإنجليزي الممتاز ، حيث لعب في صفوف الفريق الرديف ، كما لعب في صفوف الفريق الرديف أيضا.. بينما لعب سمولينغ في صفوف نادي فولهام ، الذي يلعب في دوري الدرجة الأولى الإنجليزي ، في موسم 2007 - 2008 ، حيث لعب في صفوفه حتى الآن ، حيث لعب في صفوف الفريق الرديف ، كما لعب في صفوف الفريق الرديف أيضا.. في حين لعب سمولينغ في صفوف نادي مانشستر يونايتد ، الذي يلعب في دوري الدرجة الأولى الإنجليزي ، في موسم 2008 - 2009 ، حيث لعب في صفوفه حتى الآن ، حيث لعب في صفوف الفريق الرديف ، كما لعب في صفوف الفريق الرديف أيضا.. في حين لعب سمولينغ في صفوف نادي مانشستر يونايتد ، الذي يلعب في دوري الدرجة الأولى الإنجلي

In [22]:
# Question to ask
question = 'استعادت جدّة إماراتية في الثانية والتسعين من العمر عافيتها، وكسبت فرصة جديدة للعيش، بعدما نجح الأطباء في العاصمة الإماراتية في إجراء عملية جراحية طارئة لإنقاذها من نوبة قلبية خطيرة ومضاعفات أخرى.'
# Generate the final answer using AraGPT2
answer = rag_chain(example_context, question)
print(answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


هنا يوجد مثال على السياق الذي يحتوي على معلومات ذات صلة بسؤال المستخدم.

السؤال: استعادت جدّة إماراتية في الثانية والتسعين من العمر عافيتها، وكسبت فرصة جديدة للعيش، بعدما نجح الأطباء في العاصمة الإماراتية في إجراء عملية جراحية طارئة لإنقاذها من نوبة قلبية خطيرة ومضاعفات أخرى.
الإجابة: في عام 2006 ، كانت الطفلة التي تبلغ من العمر خمس سنوات ، قد خضعت لعملية جراحية في القلب ، حيث كانت تعاني من قصور في القلب ، مما أدى إلى توقف القلب عن العمل ، مما أدى إلى وفاتها........................................................................................................................................................................................................................................................................................................................................................................................................


In [23]:
# Question to ask
question = ' ؟عندما التقى حمد سعيد سلطان الشامسي سفير الدولة لدى بيروت، أمس، وائل أبو فاعور وزير الصحة اللبناني. وناقش الطرفان خلال اللقاء الذي عقد بمقر وزارة الصحة اللبنانية، ماذا ناقش'
# Generate the final answer using AraGPT2
answer = rag_chain(example_context, question)
print(answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


هنا يوجد مثال على السياق الذي يحتوي على معلومات ذات صلة بسؤال المستخدم.

السؤال:  ؟عندما التقى حمد سعيد سلطان الشامسي سفير الدولة لدى بيروت، أمس، وائل أبو فاعور وزير الصحة اللبناني. وناقش الطرفان خلال اللقاء الذي عقد بمقر وزارة الصحة اللبنانية، ماذا ناقش
الإجابة: م. محمد أبو العينين ، وزير الصحة الفلسطيني ، مع نظيره اللبناني ، الدكتور جواد خليفة ، حول سبل تعزيز التعاون الصحي بين البلدين.


In [24]:
# Question to ask
question = 'ناقش قضية أطفال الشوارع أنه من الناحية النفسية فإن أطفال الشوارع يصابون بالقلق والتبول اللاإرادي إلى جانب بعض الأمراض النفسية الأخرى مثل الاكتئاب والعصبية والحقد على المجتمع، لأنهم يشعرون بعدم الأمان والظلم، أما من الناحية الجسدية'

# Generate the final answer using AraGPT2
answer = rag_chain(example_context, question)
print(answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


هنا يوجد مثال على السياق الذي يحتوي على معلومات ذات صلة بسؤال المستخدم.

السؤال: ناقش قضية أطفال الشوارع أنه من الناحية النفسية فإن أطفال الشوارع يصابون بالقلق والتبول اللاإرادي إلى جانب بعض الأمراض النفسية الأخرى مثل الاكتئاب والعصبية والحقد على المجتمع، لأنهم يشعرون بعدم الأمان والظلم، أما من الناحية الجسدية
الإجابة: في حين أن الأطفال الذين يعانون من مشاكل في النمو العقلي يعانون من مشاكل في النمو الجسدي ، فإن الأطفال الذين يعانون من مشاكل في النمو الجسدي يعانون من مشاكل في النمو الجسدي................................................................................................................................................................................................................................................................................................................................................................................................................


vector store

**user friendly interface**

In [29]:
!pip install googledriver
!pip install sentence_transformers
!pip install langchain
!pip install langchain_community
!pip install chromadb
!pip install langchain_huggingface
!pip install langchain_chroma
!pip install langchain_google_genai
!pip install langchain_core
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 13.0.1
    Uninstalling websockets-13.0.1:
      Successfully uninstalled websockets-13.0.1
  Attempting uninstall: tomlkit
    Found existing installation: tomlkit 0.13.2
    Uninstalling tomlkit-0.13.2:
      Successfully uninstalled tomlkit-0.13.2


In [16]:
# Install required libraries
# !pip install gradio transformers langchain gdown

import os
import gdown
import gradio as gr
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate

# Define the directory path and ensure it exists
folder_path = '/content/db22'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    # Download and extract data from Google Drive
    url = 'https://drive.google.com/drive/folders/1shvLz8dkNxAF9eyjtuFS0VG9E-ZzsGy2?usp=sharing'
    gdown.download_folder(url, quiet=False, use_cookies=False)
    !unzip /content/database_db2/db22.zip -d /content/db22



# Initialize the embeddings and Chroma vector store
model_name = "Omartificial-Intelligence-Space/GATE-AraBert-v1"
embeder = HuggingFaceEmbeddings(model_name=model_name)
persist_directory = '/content/db22'
db2 = Chroma(embedding_function=embeder, persist_directory=persist_directory)

# Initialize retriever
retriever = db2.as_retriever(search_type="similarity", search_kwargs={"k": 5})

# Initialize the AraGPT2 model
aragpt2_model_name = "aubmindlab/aragpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(aragpt2_model_name)
aragpt2_model = aragpt2_model = GPT2LMHeadModel.from_pretrained(aragpt2_model_name, max_length=1000)
# Define the prompt template
template = '''
You are an AI chatbot developed by humans, and your task is to answer user questions based on the knowledge provided in the context below.
The question will be in Arabic, and your answer should be in Arabic as well.
The provided context contains information about various categories: [Culture, Finance, Medicine, Politics, Religion, Sports, and Technology].
Try to answer using the information given in the context below. If the answer is not clearly available in the context, follow these steps:
    1. Apologize to the user.
    2. Inform the user that you do not know the answer to the question.
    3. Ask the user if they have any more questions.
At the end of the answer, thank the user and ask if they have any other questions.
Your answer should be detailed and at least 300 words long.
If asked about yourself, tell the user that you are a chatbot developed by humans and not a human.

Knowledge you have:
{context}

Question: {question}

Answer:
'''
example_context = "هنا يوجد مثال على السياق الذي يحتوي على معلومات ذات صلة بسؤال المستخدم."
def format_docs(docs):
    """Format documents into a single string."""
    return "\n\n".join(docs)  # Assuming 'docs' is a list of strings

def generate_aragpt2_response(context, question):
    input_text = f"{context}\n\nالسؤال: {question}\nالإجابة:"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    output = aragpt2_model.generate(input_ids, max_length=500, num_return_sequences=1)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text


def answer_question(question):
    """Process the input question to generate related text and answer."""
    # Retrieve documents from the retriever
    try:
        retrieved_docs = retriever.get_relevant_documents(question)
    except Exception as e:
        print(f"Error retrieving documents: {e}")
        return "Error retrieving documents.", "An error occurred during generation. Please check the input or reduce its length."

    # Check if documents are retrieved
    if not retrieved_docs:
        return "No relevant documents found.", "An error occurred during generation. Please check the input or reduce its length."

    # Extract and format document contents
    nearest_text = format_docs([doc.page_content for doc in retrieved_docs])

    # Check if nearest_text is empty or too large
    if not nearest_text.strip():
        return "Retrieved content is empty.", "An error occurred during generation. Please check the input or reduce its length."

    # Ensure the nearest_text is within the token length limits
    max_tokens_context = tokenizer.model_max_length - 200  # Reserve tokens for the question and response
    formatted_context = nearest_text[:max_tokens_context]  # Truncate if needed

    # Generate the answer using the AraGPT2 model
    try:
        answer = generate_aragpt2_response(example_context, question)
    except IndexError as e:
        print("IndexError during response generation:", e)
        return formatted_context, "An error occurred during generation. Please check the input or reduce its length."

    return formatted_context, answer






In [18]:
# Test the updated function
question = 'ناقش قضية أطفال الشوارع أنه من الناحية النفسية فإن أطفال الشوارع يصابون بالقلق والتبول اللاإرادي إلى جانب بعض الأمراض النفسية الأخرى مثل الاكتئاب والعصبية والحقد على المجتمع، لأنهم يشعرون بعدم الأمان والظلم، أما من الناحية الجسدية'

# Generate the final answer using AraGPT2
nearest_text, answer = answer_question(question)
print("Nearest Text:", nearest_text)
print("Answer:", answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Nearest Text: بعضهم في مراكز إيواء وتعليمهم مهنا مختلفة أو إلحاقهم بمدارس مفتوحة بالإضافة إلى الكشف عن العوامل البيئية -والأسرية والاجتماعية- التي أدت إلى تنامي هذه الظاهرة.مرض العزلة الاجتماعيةالدكتور أحمد البحيري استشاري الطب النفسي بالأمانة العامة للصحة النفسية يشير الى أنه من الناحية النفسية فإن أطفال الشوارع يصابون بالقلق والتبول اللاإرادي إلى جانب بعض الأمراض النفسية الأخرى مثل الاكتئاب والعصبية والحقد على المجتمع، لأنهم يشعرون بعدم الأمان والظلم، أما من الناحية الجسدية فهؤلاء الأطفال معرضون لحوادث سيارات أو اغتصاب أو تعلم عادات سيئة مثل التسول والسرقة وإدمان المخدرات.وأكد د. البحيري أن أطفال الشوارع بالرغم من أنهم يعيشون في المدن الكبيرة إلا أن معظمهم مصابون بمرض العزلة الاجتماعية الذي يعد من أخطر الأمراض التي تهدد المجتمعات الحديثة في العالم، لأنه يؤدي إلى انحسار الشخص في نفسه ويشعر بأنه مهمش ولا قيمة له في الدنيا ويفقد التخطيط للمستقبل ويسير في الشوارع بلا هدف ويفقد علاقته بالمجتمع ويعيش في جماعات وعصابات تلتزم بأخلاقيات الشارع وبالتالي يشكل خطورة كبيرة على المجتمع.الحل كما ير

In [19]:
iface = gr.Interface(
    fn=answer_question,
    inputs="text",
    outputs=["text", "text"],  # Adjust output types if needed
    title="Arabic Question Answering with RAG",
    description="Ask a question in Arabic and receive the most relevant text and a generated answer based on the context."
)
# Launch the Gradio interface
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5811c10860975182a5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
